In [1]:
import ase.io.castep as castep

import ase
import ase.calculators.castep
import ase.io.castep
from ase.io import write
import shutil,os
calc = ase.calculators.castep.Castep()
calc._prepare_input_only = True
nos = #list of prepare initial and final states for a given barrier
from ase.visualize import view

In [3]:
#writes castep neb files uses ase to write the param file then writes the cell file having used ase neb to do the interpolation

for no in nos :
    
    initial = castep.read_castep_geom(open("barriers/saddle{}initial/castep.geom".format(no),'r'))
    initial2 = castep.read_castep_geom(open("barriers/saddle{}final/castep.geom".format(no),'r'))
    
    initial[-1].calc=calc
    calc.param.task = "TransitionStateSearch"
    calc.param.opt_strategy =  "speed"
    calc.param.max_scf_cycles = 360
    calc.param.XC_FUNCTIONAL = "RPBE"
    calc.param.MIXING_SCHEME = "Pulay"
    calc.param.NEXTRA_BANDS = 112
    calc.param.cut_off_energy = 900
    calc.param.mix_history_length= 32
    calc.param.mix_cut_off_energy = "500 eV"
    calc.param.elec_energy_tol  =  "0.5e-6 eV"
    calc.param.tssearch_method = "neb"
    calc.param.tssearch_max_path_points = 5
    calc.param.tssearch_neb_method = "ode12r"
    calc.param.tssearch_neb_max_iter = 20
    calc.param.tssearch_energy_tol = "0.22e-4 eV"
    calc.param.rand_seed = 5758057
    calc.param.iprint = 2
    calc.param.num_farms = 5

    calc.initialize()
    os.rename("CASTEP/castep.param","CASTEP/neb.param")
    os.rename("CASTEP","barrier{}".format(no))
    
    initial2[-1].calc=calc
    calc.initialize()
    shutil.move("CASTEP/castep.cell","barrier{}/castepfinal.cell".format(no))
    shutil.rmtree("CASTEP")

    from ase.neb import NEB

    atoms = initial[-1]
    final = initial2[-1]

    images = [atoms]
    images += [atoms.copy() for i in range(5)]
    images += [final]

    neb = NEB(images)

    neb.interpolate()

    images[3].calc = calc
    calc.initialize()
    shutil.move("CASTEP/castep.cell","barrier{}/castepmid.cell".format(no))
    shutil.rmtree("CASTEP")

    f1 = open("barrier{}/neb.cell".format(no), "w")


    g1 = open("barrier{}/castep.cell".format(no) , "r")
    g2 = open("barrier{}/castepfinal.cell".format(no) , "r")
    g3 = open("barrier{}/castepmid.cell".format(no) , "r")


    f1.write("%block lattice_abc \n  14.46482773 14.46482773 14.46482773 #lattice params \n 90.0  90.0  90.0 \n \
    %endblock lattice_abc \n \
    fix_all_cell : true \n \
    %BLOCK POSITIONS_ABS \n")

    glines1 = g1.readlines()
    glines1 = glines1[12:]
    glines2 = g2.readlines()
    glines2 = glines2[12:-2]
    glines3 = g3.readlines()
    glines3 = glines3[12:-2]

    #print(glines1)
    f1.write(' '.join(glines1))

    f1.write("%block positions_abs_product \n \
    ang \n")

    f1.write(' '.join(glines2))

    f1.write("%endblock positions_abs_product \n")

    f1.write("%block positions_abs_intermediate \n \
    ang \n")

    f1.write(' '.join(glines3) + "%endblock positions_abs_intermediate \n \
    kpoint_mp_grid 1 1 1 \n \
    fix_com : false")

